In [1]:
import numpy as np
import pandas as pd
from collections import namedtuple
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.cross_validation import cross_val_score
from sklearn.feature_extraction.text import HashingVectorizer
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

## Идея
В этом ноутбуке использовался gensim doc2vec и линейная регрессия. А именно, из urlов и title-ов делался bag of words для каждого id, затем это запихивалось в doc2vec, а на полученных векторах делалась линейная регрессия. К сожалению эта модель не сработала, она была хуже даже sample_solution, но зато сработал вариант c TfidfVectorizer и линейной регрессией(см. второй ipynb)

## Подготовка данных

In [2]:
# urls_train
urls_train_df = pd.read_csv('kaggle_data/url_domain_train', header=None, delimiter='\t')
urls_train_df.columns = ['id', 'url', 'count']
urls_train_df = urls_train_df[['id', 'url']]
# urls_test
urls_test_df = pd.read_csv('kaggle_data/url_domain_test', header=None, delimiter='\t')
urls_test_df.columns = ['id', 'url', 'count']
urls_test_df = urls_test_df[['id', 'url']]

In [3]:
urls_train_df.head()
urls_test_df.head()

,id,url
0,0000000151004FF4ADD746DA10685A01,afisha.ru
1,0000000151004FF4ADD746DA10685A01,aif.ru
2,0000000151004FF4ADD746DA10685A01,aimfar.solution.weborama.fr
3,0000000151004FF4ADD746DA10685A01,alkotest.ru
4,0000000151004FF4ADD746DA10685A01,aptekamos.ru


In [4]:
# urls_train
urls_train_df = pd.DataFrame(urls_train_df.groupby('id')['url'].apply(lambda x: x.tolist()))
urls_train_df['id'] = urls_train_df.index
urls_train_df.index = range(len(urls_train_df))
urls_train_df.columns = ['urls', 'id']
#urls_test
urls_test_df = pd.DataFrame(urls_test_df.groupby('id')['url'].apply(lambda x: x.tolist()))
urls_test_df['id'] = urls_test_df.index
urls_test_df.index = range(len(urls_test_df))
urls_test_df.columns = ['urls', 'id']

In [5]:
urls_train_df.head()
urls_test_df.head()

,urls,id
0,"[1000bankov.ru, 1tv.ru, 4put.ru, argumenti.ru,...",000000014A02348E701552980349FF01
1,"[autorambler.ru, bilettorg.ru, dsol-druzhba.ru...",000000014A10EA183BF8594A0B2AB201
2,"[photosight.ru, rambler.ru]",000000014A4FE5C33A929D4C26943601
3,"[base.consultant.ru, dogovor-obrazets.ru, fd.r...",000000014B7BB9957784A9BC0AC9F401
4,"[assessor.ru, audit-it.ru, base.garant.ru, com...",000000014C7749F896D82C2B01E8B801


In [6]:
age_train_df = pd.read_csv('kaggle_data/age_profile_train', header=None, delimiter='\t')
age_train_df.columns = ['id', 'age']

In [7]:
age_train_df.head()

,id,age
0,000000013CB5719C0000A2C90002C101,53
1,00000001442BE24000001B7D00F50801,48
2,00000001448580F800003F1B31FB0901,28
3,0000000145BDB2FF000157971645E901,44
4,000000014602771F0000DB9359714C01,48


In [8]:
train_df = urls_train_df.merge(age_train_df, on='id', how='left')

In [9]:
train_df.head()

,urls,id,age
0,"[id.rambler.ru, mail.rambler.ru, r0.ru]",000000013CB5719C0000A2C90002C101,53
1,"[1prime.ru, autorambler.ru, chellak.ru, docs.c...",00000001442BE24000001B7D00F50801,48
2,[bosch-korolev.ru],00000001448580F800003F1B31FB0901,28
3,"[aptekanizkihcen.ua, colady.ru, gorod.dp.ua, i...",0000000145BDB2FF000157971645E901,44
4,"[astrorok.ru, diets.ru, edaplus.info, eshzdoro...",000000014602771F0000DB9359714C01,48


In [10]:
title_train_df = pd.read_csv('kaggle_data/title_unify_train', header=None,delimiter='\t')
title_test_df = pd.read_csv('kaggle_data/title_unify_test', header=None,delimiter='\t')

In [11]:
# title train
title_train_df.columns = ['id', 'title', 'count']
title_train_df = title_train_df[['id', 'title']]
# title test
title_test_df.columns = ['id', 'title', 'count']
title_test_df = title_test_df[['id', 'title']]
title_train_df.head()
title_test_df.head()

,id,title
0,0000000151790DCC1E8322AF0B6FA701,20-километровый амур китай мост недвижимость п...
1,0000000151790DCC1E8322AF0B6FA701,24-х 34-х до договор неделя новость предложить...
2,0000000151790DCC1E8322AF0B6FA701,3xl armour compress heatgear long size sleev s...
3,0000000151790DCC1E8322AF0B6FA701,4-е ca зачёт медальный место новость ои-2012 п...
4,0000000151790DCC1E8322AF0B6FA701,4-колёсный moi-bebik oregon oscar ru коляска к...


In [12]:
# title train
title_train_df = pd.DataFrame(title_train_df.groupby('id')['title'].apply(lambda f: f.tolist()))
title_train_df.columns = ['titles']
# title test
title_test_df = pd.DataFrame(title_test_df.groupby('id')['title'].apply(lambda f: f.tolist()))
title_test_df.columns = ['titles']

In [13]:
# title train
title_train_df['id'] = title_train_df.index
title_train_df.index = range(len(title_train_df))
title_train_df.columns = ['titles', 'id']
# title test
title_test_df['id'] = title_test_df.index
title_test_df.index = range(len(title_test_df))
title_test_df.columns = ['titles', 'id']
title_train_df.head()
title_test_df.head()

,titles,id
0,[11-й гектар дом коммунизм на набережная недос...,000000014A02348E701552980349FF01
1,[20-летний выглядеть девушка как королева ната...,000000014A10EA183BF8594A0B2AB201
2,[медийный портал рамблер],000000014A4FE5C33A929D4C26943601
3,"[бесплатно код онлайн срочно статистика, или и...",000000014B7BB9957784A9BC0AC9F401
4,[bank privat автокредитование банковский банко...,000000014C7749F896D82C2B01E8B801


In [14]:
train_df = title_train_df.merge(train_df,on='id')
test_df = title_test_df.merge(urls_test_df,on='id')
# train_df = train_df[['id', 'titles', 'urls', 'age']]
train_df.head()
test_df.head()

,titles,id,urls
0,[11-й гектар дом коммунизм на набережная недос...,000000014A02348E701552980349FF01,"[1000bankov.ru, 1tv.ru, 4put.ru, argumenti.ru,..."
1,[20-летний выглядеть девушка как королева ната...,000000014A10EA183BF8594A0B2AB201,"[autorambler.ru, bilettorg.ru, dsol-druzhba.ru..."
2,[медийный портал рамблер],000000014A4FE5C33A929D4C26943601,"[photosight.ru, rambler.ru]"
3,"[бесплатно код онлайн срочно статистика, или и...",000000014B7BB9957784A9BC0AC9F401,"[base.consultant.ru, dogovor-obrazets.ru, fd.r..."
4,[bank privat автокредитование банковский банко...,000000014C7749F896D82C2B01E8B801,"[assessor.ru, audit-it.ru, base.garant.ru, com..."


## Снижение размерности

In [15]:
X = np.array(list(map(lambda x: x[0] + x[1],zip(train_df.urls.values,train_df.titles.values))))
X_test = np.array(list(map(lambda x: x[0] + x[1],zip(test_df.urls.values,test_df.titles.values))))

In [16]:
y = train_df.age.values

In [17]:
Z = list(map(lambda x: list(map(lambda y: y.split(" "), x)), X)) # split internal phrases into words
Z_test = list(map(lambda x: list(map(lambda y: y.split(" "), x)), X_test))

In [18]:
T = list(map(lambda l :[item for sublist in l for item in sublist], Z)) # flatten internal lists of lists
T_test = list(map(lambda l :[item for sublist in l for item in sublist], Z_test))

In [19]:
Y = list(map(lambda x: TaggedDocument(words=x[1],tags=[x[0]]),enumerate(list(T)))) # create TaggedDocuments for doc2vec
Y_test = list(map(lambda x: TaggedDocument(words=x[1],tags=[len(Y) + x[0]]),enumerate(list(T_test))))

In [66]:
print(np.array(Y))

[[ ['id.rambler.ru', 'mail.rambler.ru', 'r0.ru', 'бесплатный', 'надёжный', 'почта', 'рамблер', 'электронный', 'поиск', 'рамблер', 'профиль', 'чемпионат']
  [0]]
 [ ['1prime.ru', 'autorambler.ru', 'chellak.ru', 'docs.cntd.ru', 'echo.msk.ru', 'expert.ru', 'finance.rambler.ru', 'forbes.ru', 'forum.ixbt.com', 'garant.ru', 'govoritmoskva.ru', 'kommersant.ru', 'kp.ru', 'lenta.ru', 'mait.ru', 'metronews.ru', 'mk.ru', 'news.rambler.ru', 'news.smi2.ru', 'norm-load.ru', 'pfr.kirov.ru', 'pfrf.ru', 'photography-on-the.ru', 'realty.rambler.ru', 'ren.tv', 'riafan.ru', 'rns.online', 'rossbanki.ru', 'secretmag.ru', 'tehnorma.ru', 'tiu.ru', 'top68.ru', 'tvc.ru', 'tvzvezda.ru', 'vesti.ru', 'video.rambler.ru', 'weekend.rambler.ru', '24-х', '34-х', 'до', 'договор', 'неделя', 'новость', 'предложить', 'рабочий', 'рамблер', 'россиянин', 'сократить', 'трудовой', 'часы', 'al-hayat', 'new', 'rambler', 'servic', 'асад', 'договорённость', 'из', 'между', 'отъезд', 'рф', 'сирия', 'сообщить', 'сша', 'al-hayat', 'аса

In [20]:
doc2vec = Doc2Vec(Y + Y_test, size=1000, window=8, min_count=5, workers=4)

In [21]:
np.array(doc2vec.docvecs).shape

(134045, 1000)

In [22]:
C = np.array(doc2vec.docvecs)[:len(Y)]
C_test = np.array(doc2vec.docvecs)[len(Y):]

## Обучение модели

In [23]:
reg = LinearRegression()
- cross_val_score(reg, C, y, scoring='mean_squared_error')

array([ 156.78158219,  158.72334977,  139.46715202])

## Отправка Решения

In [24]:
reg = LinearRegression()
reg.fit(C, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [25]:
y_pred = reg.predict(C_test)

In [28]:
test_df['age'] = y_pred

In [30]:
test_df = test_df[['id', 'age']]
test_df.columns = ['Id', 'age']

In [31]:
test_df.head()

,Id,age
0,000000014A02348E701552980349FF01,34.695022
1,000000014A10EA183BF8594A0B2AB201,38.838226
2,000000014A4FE5C33A929D4C26943601,36.704433
3,000000014B7BB9957784A9BC0AC9F401,35.442907
4,000000014C7749F896D82C2B01E8B801,37.265491


In [33]:
random_sol = pd.read_csv('random_solution.csv')
miss_idx = set(random_sol.Id.values) - set(test_df.Id.values)
miss_df = pd.DataFrame(list(zip(list(miss_idx), np.ones(len(miss_idx)))))
miss_df.columns = ['Id', 'age']

In [34]:
test_df = test_df.append(miss_df, ignore_index=True)

In [35]:
test_df.to_csv('solution.csv', index=False)

In [36]:
!wc -l solution.csv

   19980 solution.csv
